In [1]:
# Load required packages
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyodbc

In [2]:
# Create connection to AdventureWorksDW
conn = pyodbc.connect('Driver={SQL Server};'
                        'Server=localhost\SQLEXPRESS;'
                        'Database=AdventureWorksDW2019;'
                        'Trusted_Connection=yes;')

In [3]:
# Define sql to query SalesView
sql = "SELECT * FROM SalesView"

# Load SalesView
df = pd.read_sql(sql, conn)
df.head()

C:\Users\JoeRatterman\AppData\Local\Temp\ipykernel_12492\1654715154.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,OrderYear,OrderMonth,OrderDate,OrderDateKey,CustomerKey,SalesTerritoryKey,SalesAmount,CountryRegionCode,StateProvinceCode,ProductKey,ProductAlternateKey,EnglishProductName
0,2010,12,2010-12-29,20101229,21768,6,3578.2700,CA,BC,310,BK-R93R-62,"Road-150 Red, 62"
1,2010,12,2010-12-29,20101229,28389,7,3399.9900,FR,93,346,BK-M82S-44,"Mountain-100 Silver, 44"
2,2010,12,2010-12-29,20101229,25863,1,3399.9900,US,OR,346,BK-M82S-44,"Mountain-100 Silver, 44"
3,2010,12,2010-12-29,20101229,14501,4,699.0982,US,CA,336,BK-R50B-62,"Road-650 Black, 62"
4,2010,12,2010-12-29,20101229,11003,9,3399.9900,AU,NSW,346,BK-M82S-44,"Mountain-100 Silver, 44"


In [4]:
# Clean column names
df.rename(
    columns = {
        "OrderYear": "year",
        "OrderMonth": "month",
        "SalesAmount": "sales", 
        "ProductKey": "item", 
        "SalesTerritoryKey": "sales_terr_key"
    }
    , inplace = True 
)

In [5]:
# Add day and day of week to df
df["day"] = pd.to_datetime(df["OrderDate"]).dt.day
df["day_of_week"] = pd.to_datetime(df["OrderDate"]).dt.day_of_week
df.head()

,year,month,OrderDate,OrderDateKey,CustomerKey,sales_terr_key,sales,CountryRegionCode,StateProvinceCode,item,ProductAlternateKey,EnglishProductName,day,day_of_week
0,2010,12,2010-12-29,20101229,21768,6,3578.2700,CA,BC,310,BK-R93R-62,"Road-150 Red, 62",29,2
1,2010,12,2010-12-29,20101229,28389,7,3399.9900,FR,93,346,BK-M82S-44,"Mountain-100 Silver, 44",29,2
2,2010,12,2010-12-29,20101229,25863,1,3399.9900,US,OR,346,BK-M82S-44,"Mountain-100 Silver, 44",29,2
3,2010,12,2010-12-29,20101229,14501,4,699.0982,US,CA,336,BK-R50B-62,"Road-650 Black, 62",29,2
4,2010,12,2010-12-29,20101229,11003,9,3399.9900,AU,NSW,346,BK-M82S-44,"Mountain-100 Silver, 44",29,2


In [6]:
# Group sales by day by item by store
sales_df = df.groupby(["sales_terr_key", "item", "OrderDate", "year", "month", "day", "day_of_week"], as_index = False)["sales"].sum()
sales_df.sort_values(["sales_terr_key", "item", "year", "month", "day", "day_of_week"]).head()

,sales_terr_key,item,OrderDate,year,month,day,day_of_week,sales
0,1,214,2013-01-12,2013,1,12,5,34.99
1,1,214,2013-01-27,2013,1,27,6,34.99
2,1,214,2013-01-30,2013,1,30,2,34.99
3,1,214,2013-02-03,2013,2,3,6,34.99
4,1,214,2013-02-04,2013,2,4,0,104.97


In [7]:
# Filter for 2013 data
sales_df = sales_df[(sales_df["OrderDate"] <= '2013-12-31') & (sales_df["OrderDate"] >= '2013-01-01')]

# Convert OrderDate to datetime
sales_df["OrderDate"] = pd.to_datetime(sales_df["OrderDate"])

In [8]:
# Get unique list of territories & items
unique_val_df = sales_df[["sales_terr_key", "item"]].drop_duplicates()

# Add join key
unique_val_df['key'] = 0
unique_val_df.head()

,sales_terr_key,item,key
0,1,214,0
193,1,217,0
398,1,222,0
630,1,225,0
830,1,228,0


In [9]:
# Create full date range df
date_df = pd.DataFrame(pd.date_range(min(sales_df["OrderDate"]), pd.to_datetime('2014-01-15')), columns = ["OrderDate"])

# Add join key
date_df['key'] = 0

In [10]:
# Merge items w/ dates
full_date_df = date_df.merge(unique_val_df, how = "outer", on = 'key').sort_values(["sales_terr_key", "item", "OrderDate"])
full_date_df.head()

,OrderDate,key,sales_terr_key,item
0,2013-01-01,0,1,214
764,2013-01-02,0,1,214
1528,2013-01-03,0,1,214
2292,2013-01-04,0,1,214
3056,2013-01-05,0,1,214


In [11]:
# Create full date range sales_df
final_sales_df = full_date_df.merge(sales_df, how = "left", on = ['sales_terr_key', 'item', 'OrderDate'])

# Clean sales NA values
final_sales_df["sales"] = final_sales_df["sales"].fillna(0)

# Keep required columns
final_sales_df = final_sales_df[["OrderDate", "sales_terr_key", "item", "sales"]]
final_sales_df.tail()

,OrderDate,sales_terr_key,item,sales
290315,2014-01-11,10,606,0.0
290316,2014-01-12,10,606,0.0
290317,2014-01-13,10,606,0.0
290318,2014-01-14,10,606,0.0
290319,2014-01-15,10,606,0.0


In [12]:
# Calculate rolling average
final_sales_df["rolling_sales_pred"] = final_sales_df.groupby(['sales_terr_key', 'item'])['sales'].transform(lambda x: x.rolling(15, 1).mean())

In [13]:
# Calculate 15 days ago
min_date = max(final_sales_df["OrderDate"]) - datetime.timedelta(15)

# Filter final_sales_df for last 15 days
results_df = final_sales_df[final_sales_df["OrderDate"] > min_date]

# Change OrderDate type
results_df["OrderDate"] = results_df["OrderDate"].dt.date

# Select final columns
results_df = results_df[["OrderDate", "sales_terr_key", "item", "rolling_sales_pred"]]
results_df.head()

C:\Users\JoeRatterman\AppData\Local\Temp\ipykernel_12492\195156367.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df["OrderDate"] = results_df["OrderDate"].dt.date


,OrderDate,sales_terr_key,item,rolling_sales_pred
365,2014-01-01,1,214,41.988000
366,2014-01-02,1,214,34.990000
367,2014-01-03,1,214,32.657333
368,2014-01-04,1,214,27.992000
369,2014-01-05,1,214,25.659333


In [27]:
results_df.dtypes

OrderDate              object
sales_terr_key          int64
item                    int64
rolling_sales_pred    float64
dtype: object

In [30]:
# Create python cursor
cursor = conn.cursor()

# Drop results table
cursor.execute("DROP TABLE ForecastTable")

# Create results table if needed
cursor.execute(
    
""" 
CREATE TABLE ForecastTable(
    OrderDate            VARCHAR(50) NOT NULL
    ,sales_terr_key      VARCHAR(4) NOT NULL
    ,item                VARCHAR(30) NOT NULL
    ,rolling_sales_pred  NUMERIC(10,5) NOT NULL
    )
    
"""
)

In [29]:

# Insert Dataframe into SQL Server:
for i in range(len(results_df)):
    print(results_df.iloc[i])
    cursor.execute("INSERT INTO ForecastTable (OrderDate, sales_terr_key, item, rolling_sales_pred) values(?,?,?,?)"
    , results_df.iloc[i][0]
    , str(results_df.iloc[i][1])
    , str(results_df.iloc[i][2])
    , float(results_df.iloc[i][3])
    )
conn.commit()

OrderDate             2014-01-01
sales_terr_key                 1
item                         214
rolling_sales_pred        41.988
Name: 365, dtype: object


Error: ('HYC00', '[HYC00] [Microsoft][ODBC SQL Server Driver]Optional feature not implemented (0) (SQLBindParameter)')

In [133]:
# Close database connection
cursor.close()
print('Insert Complete')

Insert Complete
